In [2]:
import vrep
import sys
import numpy as np
import matplotlib.pyplot as plt
from skimage import draw, measure, data, color
sim.simxFinish(-1)
#port 19999 doesn't work, 1999
clientID = sim.simxStart('127.0.0.1', 19997, True, True, 5000, 5)

if clientID!= -1:
    print("Connected to remote server")
    while clientID!=-1:
        errorCode=sim.simxStartSimulation(cliendID,sim.simx_opmode_oneshot_wait)
        errorCode,cam=sim.simxGetObjectHandle(clientID,'observeEnvir',sim.simx_opmode_oneshot_wait)
        errorCode,resolution,image=sim.simxGetVisionSensorImage(clientID, cam, 0, sim.simx_opmode_oneshot_wait)
        pedaliKruti=0
        #Convertaion string image to array
        src =np.array(image, np.uint8)
        src=np.resize(src,(resolution[0],resolution[1],3))
        rImg=color.rgb2hsv(src)[...,0]<0.05
        sims=color.rgb2hsv(src)[...,2]>0#Value component
        #Image include Hue and Value
        HnV=rImg&sims
        #measurement of colored region
        red_label=measure.label(HnV)
        for region_prop in measure.regionprops(red_label):
            if((region_prop.area/rImg.size)>0.1):
                pedaliKruti=region_prop.area/rImg.size
                print(pedaliKruti)
        #Sending signal to V-rep
        if pedaliKruti>0.18:
            velosiped=sim.simxSetIntegerSignal(clientID,'OmaevaNani',0,sim.simx_opmode_oneshot_wait)
        else:
            velosiped=sim.simxSetIntegerSignal(clientID,'OmaevaNani',5,sim.simx_opmode_oneshot_wait)
        plt.imshow(HnV)
        plt.show()

else:
    print('Connection not successful')
    sys.exit('Could not connect')


ModuleNotFoundError: No module named 'sim'